In [16]:
# Skript, ohne Löschung von NaN Werten
"""
import json

def transform_data(data):
    transformed_data = []
    
    for year, entries in data.items():
        for entry in entries:
            
            transformed_entry = {
                "name": str(entry.get("Studiengang-Drucktext", "")).strip(),
                "short": str(entry.get("Stg-Kurztext", "")).strip(),
                "faculty": entry.get("FAKsort", -1),
                "fak": str(entry.get("FAK", "")).strip(),
                "campus": str(entry.get("St-Ort", "")).strip(),
                "number": str(entry.get("Stg-Nr", "")).strip(),
                "type": str(entry.get("Abschluss", "")).strip(),
                "semester": [
                    {
                        "semester": year,
                        "data": {
                            "total": entry.get("HSB-Gesamt", 0),
                            "totalGerman": entry.get("D-Gesamt", 0),
                            "maleGerman": entry.get("M\u00e4nnlich-D", 0),
                            "femaleGerman": entry.get("Weiblich-D", 0),
                            "diverseGerman": entry.get("nichtMW-D", 0),
                            "totalForeign": entry.get("A-Gesamt", 0),
                            "maleForeign": entry.get("M\u00e4nnlich-A", 0),
                            "femaleForeign": entry.get("Weiblich-A", 0),
                            "diverseForeign": entry.get("nichtMW-A", 0),
                            "totalPause": entry.get("BeurlaubteGesamt", 0),
                            "malePause": entry.get("Beurlaubte-M", 0),
                            "femalePause": entry.get("Beurlaubte-W", 0),
                            "diversePause": entry.get("Beurlaubte-nichtMW", 0),
                            "totalBeginner": entry.get("StdAnfGesamt", 0),
                            "maleBeginner": entry.get("StdAnf-M", 0),
                            "femaleBeginner": entry.get("StdAnf-W", 0),
                            "diverseBeginner": entry.get("StdAnf-nichtMW", 0)
                        }
                    }
                ]
            }
            transformed_data.append(transformed_entry)
    
    return transformed_data

# Pfad zur Eingabedatei
input_file_path = "Fakultätsdaten.json"

# JSON-Daten aus Datei einlesen
with open(input_file_path, "r") as input_file:
    data = json.load(input_file)

# Daten transformieren
transformed_data = transform_data(data)

# Pfad zur Ausgabedatei festlegen
output_file_path = "Fakultätsdaten_gefiltert.json"

# JSON-Daten in Datei schreiben
with open(output_file_path, "w") as output_file:
    json.dump(transformed_data, output_file, indent=4)

print("Daten erfolgreich in", output_file_path, "gespeichert.")
"""

Daten erfolgreich in Fakultätsdaten_gefiltert.json gespeichert.


In [2]:
import json
import math

def is_nan(value):
    try:
        return math.isnan(float(value))
    except ValueError:
        return False

def get_value(entry, keys, default=0):
    for key in keys:
        if key in entry:
            return entry.get(key, default)
    return default

def transform_data(data):
    transformed_data = []
    deleted_entries = []

    for year, entries in data.items():
        for entry in entries:
            
            transformed_entry = {
                "name": str(entry.get("Studiengang-Drucktext", "")).strip(),
                "short": str(entry.get("Stg-Kurztext", "")).strip(),
                "faculty": get_value(entry, ["FAKsort", "FK sort"], -1),
                "fak": str(get_value(entry, ["FAK", "FK"], "")).strip(),
                "campus": str(entry.get("St-Ort", "")).strip(),
                "number": str(entry.get("Stg-Nr", "")).strip(),
                "type": str(entry.get("Abschluss", "")).strip(),
                "semester": [
                    {
                        "semester": str(year.replace(" ", "")),
                        "data": {
                            "total": entry.get("HSB-Gesamt", 0),
                            "totalGerman": entry.get("D-Gesamt", 0),
                            "maleGerman": entry.get("Männlich-D", 0),
                            "femaleGerman": entry.get("Weiblich-D", 0),
                            "diverseGerman": entry.get("nichtMW-D", 0),
                            "totalForeign": entry.get("A-Gesamt", 0),
                            "maleForeign": entry.get("Männlich-A", 0),
                            "femaleForeign": entry.get("Weiblich-A", 0),
                            "diverseForeign": entry.get("nichtMW-A", 0),
                            "totalPause": entry.get("BeurlaubteGesamt", 0),
                            "malePause": entry.get("Beurlaubte-M", 0),
                            "femalePause": entry.get("Beurlaubte-W", 0),
                            "diversePause": entry.get("Beurlaubte-nichtMW", 0),
                            "totalBeginner": get_value(entry, ["StdAnfGesamt", "StdAnf-Gesamt"], 0),
                            "maleBeginner": entry.get("StdAnf-M", 0),
                            "femaleBeginner": entry.get("StdAnf-W", 0),
                            "diverseBeginner": entry.get("StdAnf-nichtMW", 0)
                        }
                    }
                ]
            }

            # Liste der zu überprüfenden Felder
            fields_to_check = [
                transformed_entry["name"],
                transformed_entry["short"],
                transformed_entry["faculty"],
                transformed_entry["fak"],
                transformed_entry["campus"],
                transformed_entry["number"],
                transformed_entry["type"]
            ]

            # Überprüfen, wie viele Felder NaN sind
            nan_count = sum(1 for field in fields_to_check if is_nan(field))

            # Nur Einträge hinzufügen, bei denen weniger als 2 Felder NaN sind
            if nan_count < 2:
                transformed_data.append(transformed_entry)
            else:
                deleted_entries.append(transformed_entry)

    return transformed_data, deleted_entries

# Pfad zur Eingabedatei
input_file_path = "Fakultätsdaten.json"

# JSON-Daten aus Datei einlesen
with open(input_file_path, "r") as input_file:
    data = json.load(input_file)

# Daten transformieren
transformed_data, deleted_entries = transform_data(data)

# Anzahl der gelöschten Einträge ausgeben
print(f"{len(deleted_entries)} Einträge wurden gelöscht.")

# Gelöschte Einträge ausgeben
print("Gelöschte Einträge:")
for entry in deleted_entries:
    print(json.dumps(entry, indent=4, ensure_ascii=False))

# Pfad zur Ausgabedatei festlegen
output_file_path = "Fakultätsdaten_gefiltert.json"

# JSON-Daten in Datei schreiben
with open(output_file_path, "w") as output_file:
    json.dump(transformed_data, output_file, indent=4)

print("Daten erfolgreich in", output_file_path, "gespeichert.")


35 Einträge wurden gelöscht.
Gelöschte Einträge:
{
    "name": "nan",
    "short": "nan",
    "faculty": NaN,
    "fak": "nan",
    "campus": "nan",
    "number": "nan",
    "type": "nan",
    "semester": [
        {
            "semester": "SoSe2014",
            "data": {
                "total": NaN,
                "totalGerman": NaN,
                "maleGerman": NaN,
                "femaleGerman": NaN,
                "diverseGerman": 0,
                "totalForeign": NaN,
                "maleForeign": NaN,
                "femaleForeign": NaN,
                "diverseForeign": 0,
                "totalPause": NaN,
                "malePause": NaN,
                "femalePause": NaN,
                "diversePause": 0,
                "totalBeginner": NaN,
                "maleBeginner": NaN,
                "femaleBeginner": NaN,
                "diverseBeginner": 0
            }
        }
    ]
}
{
    "name": "GESAMT",
    "short": "nan",
    "faculty": NaN,
    "fak": "nan"